In [2]:
import numpy as np
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import glob
import math
import plotly.express as px

import re, string
import seaborn as sns
import pickle


from sklearn.model_selection import train_test_split
import sys
sys.path.append('../')

from src.data.feature_engineering import preprocess_train_df, tokenize
from src.conf.global_variables import DATA_PATH, SEED, LABELS, COMMENT
from src.models.models_train import fit_models
from src.models.models_predict import get_prediction
#from src.models.models_save_and_load import load_models

%load_ext autoreload
%autoreload 2

ImportError: cannot import name 'COMMENT' from 'src.conf.global_variables' (../src/conf/global_variables.py)

## Load the Model 

In [ ]:
models_filenames = glob.glob('../models/model_*.pkl')
vectorizer_filename = '../models/vectorizer.pkl'

In [ ]:
def load_models(models_filenames, vectorizer_filename):
    all_loaded_models = {}
    for idx, filename in enumerate(models_filenames):
        model_class = models_filenames[idx].split('/')[2].split('.')[0].split('model_')[1]
        all_loaded_models[model_class] = pickle.load(open(filename, 'rb'))
    vectorizer = pickle.load(open(vectorizer_filename, 'rb'))
    return all_loaded_models, vectorizer

In [ ]:
classifiers, vectorizer = load_models(models_filenames, vectorizer_filename)

## Prediction

In [ ]:
sentence = 'hey fucking bitch'

In [ ]:
preds, test_term_doc = get_prediction(sentence,vectorizer, classifiers, LABELS)

## Interpret prediction

Créons maintenant un dataframe propre avec les mots / poids

In [ ]:
def get_words_weights_df(vectorizer, test_term_doc, classifiers, label):
    words_vectorizer = vectorizer.get_feature_names()

    words = np.array(words_vectorizer)[test_term_doc.indices]
    weights = classifiers[label].coef_.ravel()[test_term_doc.indices]

    df_words_weights = pd.DataFrame({'words': words,
                                     'weights': weights})
    return df_words_weights

In [ ]:
df_words_weights = get_words_weights_df(vectorizer, test_term_doc, classifiers, 'toxic')

On peut d'abord recalculer la proba pour être sûr

In [ ]:
def sigmoid(x):
    return 1 / (1 + math.exp(-x))

In [ ]:
all_x = classifiers['toxic'].intercept_.ravel() + df_words_weights['weights'].sum()

In [ ]:
proba = sigmoid(all_x)
print(proba)

In [ ]:
def odds(x):
    return sigmoid(x) / (1 - sigmoid(x))

In [ ]:
odds_with_feature = odds(all_x)
print(odds_with_feature)

Comment on interprète ça ?

Si on enlève fucking, 

In [ ]:
sentence = 'hey bitch'

In [ ]:
preds, test_term_doc_without_features = get_prediction(sentence,vectorizer, classifiers, LABELS)

In [ ]:
df_words_weights_without_feature = get_words_weights_df(vectorizer,
                                                        test_term_doc_without_features,
                                                        classifiers,
                                                        'toxic')

In [ ]:
df_words_weights_without_feature

In [ ]:
x_without_feature = classifiers['toxic'].intercept_.ravel() + df_words_weights_without_feature['weights'].sum()

#### Compare the actual odds_without_feature and the one using the exp(weight) 

In [ ]:
odds_without_feature = odds(x_without_feature)
print(odds_without_feature)

In [ ]:
def odds_withdraw_feature(odds_with_feature, weight_feature):
    return odds_with_feature / np.exp(weight_feature)

In [ ]:
odds_withdraw_feature(odds_with_feature, df_words_weights.weights[1])

#### Compute proba without feature

In [ ]:
preds[0][0]

In [ ]:
fig = px.bar(df_words_weights.sort_values(ascending= False, by='weights'), x ='words', y='weights',
             height=400)
fig.show()

In [ ]:
df_words_weights['weights_exp'] = np.exp(df_words_weights['weights'])

fig = px.bar(df_words_weights, x ='words', y='weights_exp',
             height=400)
fig.show()

# Empty words

In [ ]:
sentence_no_words = 'petite marmotte'

In [ ]:
vectorizer.get_feature_names()

In [ ]:
preds_2, test_term_doc_2 = get_prediction(sentence_no_words,vectorizer, classifiers, LABELS)

In [ ]:
test_term_doc_2.nnz == 0

In [ ]:
sentence_empty = ''

In [ ]:
preds_empty, test_term_doc_empty = get_prediction(sentence_empty,vectorizer, classifiers, LABELS)
print(preds_empty, test_term_doc_empty)

# feature importance

In [ ]:
def get_words_weights_global_df(vectorizer, classifiers, label):
    feature_importance = classifiers[label].coef_[0]
    
    df_words = pd.DataFrame(feature_importance, vectorizer.get_feature_names()).sort_values(by=0, ascending=False)

    return df_words

In [ ]:
get_words_weights_global_df(vectorizer, classifiers, 'toxic')